In [0]:
%pylab inline

In [0]:
import dataiku
from dataiku import pandasutils as pdu
import pandas as pd

In [0]:
import dataiku
from dataiku import pandasutils as pdu
import pandas as pd
import numpy as np
import glob

#Vitrox = dataiku.Folder("Vitrox")
#VitroxS = dataiku.Folder("Vitrox_split")
#Adice = dataiku.Folder("Abnormal_dice")
#segment = dataiku.Folder("Segment")
#test = dataiku.Folder("test_augmented")
test = dataiku.Folder("tpak_axi_test")
#train = dataiku.Folder("tpak_axi_train")
#train_aug = dataiku.Folder("train_augmented")

#resnet = dataiku.Folder("google_resnet_v2")
#resnet_info=resnet.get_info()
#resnet_path=resnet_info['path']


resnet = dataiku.Folder("efficientV2")
resnet_info=resnet.get_info()
resnet_path=resnet_info['path']

#Vitrox_info=Vitrox.get_info()
#VitroxS_info=VitroxS.get_info()
#segment_info=segment.get_info()
test_info=test.get_info()
#train_info=train.get_info()
#train_aug_info=train_aug.get_info()

#Void = dataiku.Folder("Void")

#Vitrox_path=Vitrox_info['path']
#VitroxS_path=VitroxS_info['path']
#segment_path=segment_info['path']
test_path=test_info['path']
#train_path=train_info['path']
#train_aug_path=train_aug_info['path']

# Recipe outputs
out = dataiku.Folder("57FA1eZU")
out_info = out.get_info()
out_path=out_info['path']
model_path=out_path+'/effmodel2.h5'
cm_out = dataiku.Folder("cm_out")
cm_out_info = cm_out.get_info()
cm_out_path=cm_out_info['path']
path_img=cm_out_path+'/cmeffmodel2.png'
#VitroxS_path

import matplotlib.pyplot as plt
import numpy as np
import random
import tensorflow
from tensorflow import keras
from tensorflow.keras import layers
import tensorflow_datasets as tfds
import tqdm as notebook_tqdm
import splitfolders
import os
#import cv2

import os
import random
import math

import tensorflow as tf
import tensorflow_addons as tfa
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import load_img, img_to_array
import tensorflow.keras.layers as L
from tensorflow.keras.layers import Dense,Flatten,Reshape,Lambda,LeakyReLU,Dropout,\
                                    BatchNormalization,Conv1D,Conv2D,Reshape,MaxPooling2D
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.regularizers import l2
from tensorflow.keras.optimizers.schedules import ExponentialDecay
import tensorflow_hub as hub
import numpy as np
from sklearn.model_selection import train_test_split, RepeatedStratifiedKFold
from sklearn.utils import shuffle
from sklearn.metrics import accuracy_score
from tqdm import tqdm
#from tabulate import tabulate
#from pyDOE import lhs
import matplotlib.pyplot as plt
import os
from tensorflow.keras.layers import Conv2D,\
    MaxPool2D, Conv2DTranspose, Input, Activation,\
    Concatenate, CenterCrop
from tensorflow.keras.initializers import HeNormal
from tensorflow.keras.optimizers import schedules, Adam
from tensorflow.keras.losses import SparseCategoricalCrossentropy,KLDivergence
from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras.utils import plot_model
import tensorflow_datasets as tfds
from tensorflow.keras.callbacks import ReduceLROnPlateau
import matplotlib.pyplot as plt

from PIL import Image
from skimage import data, color
from skimage.transform import rescale, resize, downscale_local_mean
from skimage import io
from skimage.util import crop


import tensorflow.keras.callbacks as C

In [0]:
from keras.layers import UpSampling2D
from keras.layers import Activation
from keras.layers import MaxPool2D
from keras.layers import AveragePooling2D
from keras.layers import Add
from keras.layers import Multiply
from keras.layers import Lambda

In [0]:
from keras.callbacks import ReduceLROnPlateau, EarlyStopping

In [0]:
if tf.test.gpu_device_name():

    print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))
    tf.config.set_visible_devices([], 'GPU')  ## only this works ?!!
else:

    print("Cannot detect GPU in tensorflow")

In [0]:
batch_size = 8
img_height = 512
img_width = 512
def process(image,label):
    image = tf.cast(image/255. ,tf.float32)
    return image,label
'''train_ds = tf.keras.utils.image_dataset_from_directory(
  train_aug_path,
  validation_split=0.2,
  subset="training",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size
)

val_ds = tf.keras.utils.image_dataset_from_directory(
  train_aug_path,
  validation_split=0.2,
  subset="validation",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size
)


train_ds = train_ds.map(process)
val_ds = val_ds.map(process)

for images, labels in train_ds.take(1):
    print(images.shape)
    print(images[0])

for images, labels in val_ds.take(1):
    print(images.shape)
    print(images[0])'''
test_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)

test_path

test_ds=test_datagen.flow_from_directory(
    test_path,
    target_size=(img_height,img_width),
    color_mode='rgb',
    class_mode='categorical',
    batch_size=8,
    shuffle=True,
    seed=123,
    )
data_augmentation = keras.Sequential(
    [
        layers.experimental.preprocessing.Normalization(),
        #layers.experimental.preprocessing.Resizing(img_height, img_width),
        #layers.experimental.preprocessing.RandomFlip("horizontal_and_vertical"),
        #layers.experimental.preprocessing.RandomRotation(factor=0.02),
        #layers.experimental.preprocessing.RandomZoom(
        #    height_factor = 0.2, width_factor = 0.2
        #),
        #tf.keras.layers.RandomTranslation(height_factor=0.2, width_factor=0.2),
        #tf.keras.layers.RandomContrast(0.2),
        #layers.RandomContrast(0.2),
        #layers.RandomTranslation(height_factor=0.2, width_factor=0.2),
        #tfm.vision.preprocess_ops.color_jitter()
    ],
     name="data_augmentation",
)
# Compute the mean and the variance of the training data for normalization.
#CompleteBatchData  =next(iter(test_ds))[0]
#data_augmentation.layers[0].adapt(CompleteBatchData)

In [0]:
def ResNetClassifier(shape=(img_height, img_width, 3), n_channels=64, n_classes=2,
                      dropout=0.2, regularization=0.01):
    """
    Attention-56 ResNet
    https://arxiv.org/abs/1704.06904
    """

    regularizer = l2(regularization)

    input_ = Input(shape=shape)
    #x=data_augmentation(input_)
    #x = Conv2D(n_channels, (7, 7), strides=(2, 2), padding='same')(input_)
    #x = BatchNormalization()(x)
    #x = Activation('relu')(x)
    #x = MaxPool2D(pool_size=(3, 3), strides=(2, 2), padding='same')(x)
    resnet= tf.keras.Sequential([
            hub.KerasLayer(resnet_path,
                   trainable=True),  # Can be True, see below.
        ]
        )(input_)
    #output = Dense(1, kernel_regularizer=regularizer, activation='sigmoid')(resnet)
    output = Dense(n_classes, kernel_regularizer=regularizer, activation='softmax')(resnet)

    model = Model(input_, output)
    return model

In [0]:
model = ResNetClassifier(n_classes=2)

In [0]:
lr_reducer = ReduceLROnPlateau(monitor='val_accuracy', factor=0.2, patience=2, min_lr=10e-7, cooldown=1, verbose=1)
early_stopper = EarlyStopping(monitor='val_accuracy', min_delta=0, patience=7, verbose=1)
callbacks= [lr_reducer, early_stopper]

In [0]:
initial_learning_rate=0.00001
lr_schedule = keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate,
    decay_steps=100000,
    decay_rate=0.96,
    staircase=True)

In [0]:
model.summary()

In [0]:
#model.compile(keras.optimizers.Adam(learning_rate=0.00001), loss=tfa.losses.SigmoidFocalCrossEntropy( from_logits= False,
#    alpha = 0.25,gamma = 2.0,reduction= tf.keras.losses.Reduction.AUTO,name = 'sigmoid_focal_crossentropy'), metrics=['accuracy'])
model.compile(keras.optimizers.Adam(learning_rate=0.00001), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [0]:
#history=model.fit(train_ds,batch_size=8,epochs=50,validation_data=val_ds,callbacks=callbacks)
#history=model.fit_generator(
    #train_ds,
    #steps_per_epoch = train_ds.samples // batch_size,
    #validation_data = val_ds,
    #validation_steps = val_ds.samples // batch_size,
    #epochs = 50)

In [0]:
#model.save_weights(model_path)

In [0]:
test_ds

In [0]:
model.compile(keras.optimizers.Adam(learning_rate=0.00001), loss='categorical_crossentropy', metrics=['accuracy'])

In [0]:
model_path

model.build(input_shape =(None,img_height,img_width,3))
model.load_weights(model_path)

model

model.summary()

results=model.evaluate(test_ds, batch_size=8)

results

In [0]:
results

In [0]:
test_steps = test_ds.n // batch_size
y_pred = []
y_true = []

# Iterate over the validation data generator and predict labels in batches
for i in range(test_steps):
    X_val, y_val = test_ds.next()
    #print(y_val)
    y_pred_batch = model.predict(X_val)
    #print(np.argmax(y_pred_batch, axis=1))
    y_pred.extend(np.argmax(y_pred_batch, axis=1))
    #print(np.argmax(y_val, axis=1))
    y_true.extend(np.argmax(y_val, axis=1))

In [0]:
len(y_true)

In [0]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix
import sklearn

In [0]:
class_labels = ['Good', 'Defect']

In [0]:
# Calculate the confusion matrix
cm = confusion_matrix(y_true, y_pred, labels=[0, 1])

In [0]:
# Plot the confusion matrix
#fig=plt.imshow(cm, interpolation='nearest', cmap=plt.cm.Blues)
#plt.title('Confusion Matrix')
#plt.colorbar()
#tick_marks = np.arange(len(class_labels))
#plt.xticks(tick_marks, class_labels)
#plt.yticks(tick_marks, class_labels)

# Label the matrix cells with the counts
#thresh = cm.max() / 2
#for i, j in np.ndindex(cm.shape):
#    plt.text(j, i, format(cm[i, j], 'd'),
#             horizontalalignment="center",
#             color="white" if cm[i, j] > thresh else "black")
#
#plt.ylabel('True Label')
#plt.xlabel('Predicted Label')
#plt.tight_layout()
#plt.show()

In [0]:
import seaborn as sns
fig=sns.heatmap(cm,annot=True,fmt='g',linewidth=0.1,linecolor='black',xticklabels=['good','bad'],yticklabels=['good','bad'],
            cmap="Blues")
plt.ylabel('Prediction',fontsize=13)
plt.xlabel('Actual',fontsize=13)
plt.title('Confusion Matrix',fontsize=17)
plt.show()
plt.savefig(path_img)

In [0]:
cm

In [0]:
from sklearn.metrics import precision_score
precision = precision_score(y_true, y_pred, average='binary')
print("precision:",precision)

In [0]:
from sklearn.metrics import recall_score
recall = recall_score(y_true, y_pred, average='binary')
print("recall:",recall)

In [0]:
f1=sklearn.metrics.f1_score(y_true, y_pred)

In [0]:
print("F1 score:",f1)

In [0]:
# Recipe outputs
wmm8v6au = dataiku.Folder("wMM8v6AU")
wmm8v6au_info = wmm8v6au.get_info()